#Скрипт автоматического обнаружения факта попадания объекта в зону видимости камеры с помощью библиотеки OpenCV
1) Подготовка датасета по фреймам для обучающей и тестовой выборки
2) Обучение модели и сохрание
3) Инициализация: устанавливаются порог для обнаружения объектов и порог подавления немаксимальных значений, инициализируется вход с веб-камеры.
4) Загрузка модели.
5) Настройка модели: скрипт настраивает модель, например, устанавливает размер ввода, масштаб, среднее и меняет RB.
6) Обнаружение в реальном времени: скрипт входит в цикл, постоянно читает кадры с веб-камеры, использует модель для обнаружения объектов в каждом кадре, рисует ограничивающие рамки и метки вокруг обнаруженных объектов.
Обнаруженные объекты и их метки отображаются в реальном времени с помощью cv2_imshow().


In [1]:
# Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Установка зависимостей - подготовка окружения
#!pip install opencv-python tensorflow numpy imutils

In [ ]:
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from google.colab.patches import cv2_imshow

 пример детекции и позиционирования - автоматическое обнаружение факта попадания любого объекта в зону видимости камеры с помощью библиотеки OpenCV и предварительно обученной модели (например, SSD MobileNet)

In [ ]:
# @title создание выборки по фреймам
# путь к видеозаписи
path_video = '/content/drive/MyDrive/track001.mp4'
# чтение файла
vidcap = cv2.VideoCapture(path_video)
length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)
# Счетчик кадров
frame_count = 0
for i in range(length):
  if i == i:
  # считаем фрейм
    hasFrame, image = vidcap.read()
    # Формируем имя файла
    filename = f'/content/drive/MyDrive/dataset/train/background/frame_{frame_count}.jpg'
    # переведем BGR в RGB
    color = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Сохраняем кадр
    success = cv2.imwrite(filename, image)
    # визуализируем полученный кадр
    plt.imshow(color)
    plt.show()
  frame_count += 1

In [ ]:
# @title Обучение и сохранение модели
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Параметры
img_width, img_height = 640, 352
train_data_dir = '/content/drive/MyDrive/dataset/train/'
validation_data_dir = '/content/drive/MyDrive/dataset/test'
nb_train_samples = 1000
nb_validation_samples = 200
epochs = 25
batch_size = 32

# Создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Генераторы данных
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Обучение модели
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

# Сохранение модели
model.save('object_detection_model.h5')


Found 276 images belonging to 2 classes.
Found 15 images belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
 9/31 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.5853 - loss: 1.4203

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


31/31 ━━━━━━━━━━━━━━━━━━━━ 29s 777ms/step - accuracy: 0.6045 - loss: 1.3150 - val_accuracy: 0.6667 - val_loss: 0.6458
Epoch 2/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 326ms/step - accuracy: 0.7125 - loss: 0.6492 - val_accuracy: 0.6667 - val_loss: 0.6663
Epoch 3/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 316ms/step - accuracy: 0.7011 - loss: 0.5872 - val_accuracy: 0.6667 - val_loss: 0.6194
Epoch 4/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 22s 400ms/step - accuracy: 0.7039 - loss: 0.5865 - val_accuracy: 0.6667 - val_loss: 0.6139
Epoch 5/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 328ms/step - accuracy: 0.7053 - loss: 0.5634 - val_accuracy: 0.6667 - val_loss: 0.6182
Epoch 6/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 313ms/step - accuracy: 0.7197 - loss: 0.5405 - val_accuracy: 0.6667 - val_loss: 0.7307
Epoch 7/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 388ms/step - accuracy: 0.7356 - loss: 0.5119 - val_accuracy: 0.7333 - val_loss: 0.5663
Epoch 8/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - accuracy: 0.7836 - loss: 0.4736 - val_accuracy: 0.666

In [2]:
#@title Сохранение в файл
def save_box(ROI_number, i, original, x, y, h, w):
  x=x-5
  y=y-5
  h=h+5
  w=w+5
  ROI = original[y:y+h, x:x+w]
  if h>w: ROI =cv2.rotate(ROI, cv2.ROTATE_90_CLOCKWISE)
  cv2.imwrite(f"/content/drive/MyDrive/box/ROI_{ROI_number}_i_{i}.png", ROI)

In [7]:
#Скрипт детекции и позиционирования объектов в реальном времени
import cv2
import numpy as np
import time
from google.colab.patches import cv2_imshow

# Параметры для обработки
CONFIDENCE_THRESHOLD = 0.6
INPUT_WIDTH = 640
INPUT_HEIGHT = 352
# Инициализация камеры
#cap = cv2.VideoCapture(0)  # 0 - встроенная камера
path_video = '/content/drive/MyDrive/track001.mp4'
# изменили на путь к видеофайлу - чтение файла
cap = cv2.VideoCapture(path_video)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Цвета для визуализации
colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0)]
ROI_number = 0
for i in range(length):
#while True:
    # Чтение кадра с камеры
    ret, frame = cap.read()
    if not ret:
        break
    # Получение размеров кадра
    (H, W) = frame.shape[:2]
    # Предварительная обработка изображения
    # Изменение размера и нормализация
    image = cv2.resize(frame, (INPUT_WIDTH, INPUT_HEIGHT))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    # Начало измерения времени для расчета FPS
    start_time = time.time()
    # Обработка предсказаний
    if 1==1:
        # Поиск контуров для определения позиции объекта
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Создаем объект CLAHE для улучшения локального контраста
        #clahe = cv2.createCLAHE(clipLimit=15, tileGridSize=(8,8))
        # Применяем CLAHE к изображению
        #enhanced = clahe.apply(blurred)

        #бинаризация (0 - порогового преобразования) изображения
        #комбинация флагов: cv2.THRESH_BINARY_INV — инверсный бинарный порог, cv2.THRESH_OTSU — автоматический расчет оптимального порога
        thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Морфологическая обработка для очистки эрозия
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
        #Удаляет мелкие объекты, которые меньше структурного элемента Помогает избавиться от шума эрозия ->дилатация
        #Параметр iterations=1 означает однократное применение операции
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
        #дилатация->эрозия Закрывает небольшие отверстия внутри объектов
        #Соединяет близко расположенные объекты iterations=2 означает двукратное применение операции
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)
        colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0)]

        # Поиск контуров метод аппроксимации контуров cv2.CHAIN_APPROX_SIMPLE — сохраняются только конечные точки прямых линий
        # cv2.RETR_EXTERNAL - поиск только внешних контуров
        #входное изображение. Должно быть бинарным (обычно результат порогового преобразования или морфологических операций)
        contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
        # Фильтрация по площади
          if 600<cv2.contourArea(contour)<2800:
             x, y, w, h = cv2.boundingRect(contour)
             #область детекции с помощью координат x и y
             if 125<y<260 and (20<x<220 or 270<x<550):
               save_box(ROI_number, i, frame.copy(), x, y, h, w)
               # Определение центра
               ROI_number += 1

        # Отрисовка контуров
        for contour in contours:
        # Фильтрация по площади
          if 600<cv2.contourArea(contour)<2800:
             x, y, w, h = cv2.boundingRect(contour)
             #область детекции с помощью координат x и y
             if 125<y<260 and (20<x<220 or 270<x<550):
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # Вывод информации о позиции
                cv2.putText(frame, f"X: {x}, Y: {y}", (x, y - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[0], 2)
                M = cv2.moments(contour)
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
    #Расчет FPS и вывод фреймов
    #end_time = time.time()
    #fps = 1.0 / (end_time - start_time)
    #cv2.putText(frame, f"FPS: {fps:.2f} i: {i:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Отображение результата
    #cv2_imshow(frame)
    #cv2.imwrite(f"/content/drive/MyDrive/frame/frame_{i}.png", frame)
    # Выход по нажатию клавиши 'q'
    #if cv2.waitKey(1) & 0xFF == ord('q'): break

# Освобождение ресурсов
cap.release()
#cv2.destroyAllWindows()


# Скрипт автоматического обнаружения факта попадания объекта в зону видимости камеры с помощью YOLO
1. Резметку необходимо производить с помощью BoundigBox например в labelstudio и выгрузить images + labels
2. Обучение YOLO на собственных данных. В качестве базовой модели выбрана из версии v8 простая модель yolov8n. Так же можно увеличить сложность модели (yolov8n → yolov8m → yolov8l). Количество эпох обучения не менее 200. Количество изображений 50.
3. Конфигурация датасета (yolov8_custom.yaml):

train: ./data/train/images

val: ./data/test/images

nc: 1  # Количество классов

names: ['preform']  # Названия классов

In [12]:
# @title Установка зависимостей
!pip install ultralytics opencv-python numpy torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.9 MB/s eta 0:00:00


In [16]:
# @title Подготовка конфигурационного файла
# Создание файла yaml
%%writefile /content/drive/MyDrive/custom_yolo/runs/yolov8_custom.yaml
train: ./data/train/images
val: ./data/test/images

nc: 1 # Количество классов
names: ['preform'] # Названия классов

Writing /content/drive/MyDrive/custom_yolo/runs/yolov8_custom.yaml


In [17]:
# @title Обучение модели
from ultralytics import YOLO
#Загрузка базовой модели
#Постепенно увеличивайте сложность модели (yolov8n → yolov8m → yolov8l)
model = YOLO('yolov8n.pt')

# Настройка параметров обучения
results = model.train(
    data='/content/drive/MyDrive/custom_yolo/runs/yolov8_custom.yaml',  # Путь к конфигурации датасета
    epochs=20,                 # Количество эпох
    imgsz=[640, 352],          # Размер изображения
    batch=16,                   # Размер батча
    lr0=0.01,                   # Начальная скорость обучения
    project='runs',             # Директория для сохранения результатов
    name='exp',                 # Имя эксперимента
    exist_ok=True               # Разрешить перезапись
)

Ultralytics 8.3.218 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/custom_yolo/runs/yolov8_custom.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=[640, 352], int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

In [ ]:
# Загрузка обученной модели
model = YOLO('/content/runs/exp/weights/best.pt')

In [ ]:
# Тестирование на изображении
results = model.predict('/content/drive/MyDrive/custom_yolo/runs/data/test/images/1b170f24-frame_23.jpg')
results = model.predict('/content/drive/MyDrive/custom_yolo/runs/data/test/images/e0717e7e-frame_188.jpg')
results = model.predict('/content/drive/MyDrive/custom_yolo/runs/data/test/images/9a0ee140-frame_73.jpg')


image 1/1 /content/drive/MyDrive/custom_yolo/runs/data/test/images/1b170f24-frame_23.jpg: 352x640 1 preform, 89.8ms
Speed: 1.7ms preprocess, 89.8ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /content/drive/MyDrive/custom_yolo/runs/data/test/images/e0717e7e-frame_188.jpg: 352x640 1 preform, 89.1ms
Speed: 1.7ms preprocess, 89.1ms inference, 0.9ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /content/drive/MyDrive/custom_yolo/runs/data/test/images/9a0ee140-frame_73.jpg: 352x640 (no detections), 92.4ms
Speed: 1.8ms preprocess, 92.4ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)


In [ ]:
def detect_objects(img):
    # Загрузка изображения
    #img = cv2.imread(image_path)

    # Настройка параметров предсказания
    results = model.predict(
      img,
      conf=0.25,  # Минимальная уверенность
      iou=0.45,   # Порог пересечения
      max_det=1000  # Максимальное количество обнаружений
    )

    # Обработка результатов
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Извлечение координат bounding box
            x1, y1, x2, y2 = box.xyxy[0].tolist()

            # Извлечение класса и уверенности
            cls = int(box.cls[0])
            conf = float(box.conf[0])

            # Получение названия класса
            class_name = model.names[cls]

            # Рисование bounding box
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

            # Добавление метки класса
            cv2.putText(img, f'{class_name} {conf:.2f}', (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return img

In [ ]:
# @title Детекция и позиционирования объектов в реальном времени
import cv2
import numpy as np
import time
from google.colab.patches import cv2_imshow

# Параметры для обработки
CONFIDENCE_THRESHOLD = 0.6
INPUT_WIDTH = 640
INPUT_HEIGHT = 352
# Инициализация камеры
#cap = cv2.VideoCapture(0)  # 0 - встроенная камера
path_video = '/content/drive/MyDrive/track001.mp4'
# изменили на путь к видеофайлу - чтение файла
cap = cv2.VideoCapture(path_video)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Цвета для визуализации
colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0)]
ROI_number = 0
for i in range(length):
#while True:
    # Чтение кадра с камеры
    ret, frame = cap.read()
    if not ret:
        break
    # Получение размеров кадра
    (H, W) = frame.shape[:2]
    # Предварительная обработка изображения
    # Изменение размера и нормализация
    image = frame#cv2.resize(frame, (INPUT_WIDTH, INPUT_HEIGHT))
    #image = image / 255.0
    #image = np.expand_dims(image, axis=0)
    # Детекция объектов
    detected_image = detect_objects(image)
    # Отображение результата
    cv2_imshow(detected_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # Выход по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Освобождение ресурсов
cap.release()